In [ ]:
pip install ensemble_boxes

In [32]:
import cv2
import numpy as np
from ensemble_boxes import *
from pycocotools.coco import COCO
import pandas as pd


In [33]:
# ensemble csv files
submission_files = ['./submission/swin_base_epoch36.csv', './submission/swin_tiny_baseline_1x.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [34]:
image_ids = submission_df[0]['image_id'].tolist()

In [35]:
annotation = '../../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [36]:
prediction_strings = []
file_names = []
iou_thr = 0.4
weights = [2, 1]

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=0.00001)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [37]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble_2.csv')

submission.head()

,PredictionString,image_id
0,8 0.9994210600852966 173.4236297607422 234.466...,batch_01_vt/0021.jpg
1,8 0.9996840953826904 157.92739868164062 59.410...,batch_01_vt/0028.jpg
2,1 0.9864273071289062 70.92247009277344 168.450...,batch_01_vt/0031.jpg
3,8 1.1818219423294067 32.42074966430664 140.418...,batch_01_vt/0032.jpg
4,8 0.9996082186698914 150.2918701171875 0.15694...,batch_01_vt/0070.jpg


In [ ]:
# i = 0
# iou_thr = 0.5
# skip_box_thr = 0.0001
# sigma = 0.1
# weights = None
# boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
# boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
# boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
# boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)


# boxes0, scores0, labels0 = nms([whole_bbox[i]], [whole_prop[i]], [whole_label[i]], weights=[0, 1], iou_thr=iou_thr)
# boxes1, scores1, labels1 = soft_nms([whole_bbox[i]], [whole_prop[i]], [whole_label[i]], weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
# boxes2, scores2, labels2 = non_maximum_weighted([whole_bbox[i]], [whole_prop[i]], [whole_label[i]], weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)
# boxes3, scores3, labels3 = weighted_boxes_fusion([whole_bbox[i]], [whole_prop[i]], [whole_label[i]], weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)

In [ ]:
# whole_bbox = []
# whole_label = []
# whole_prop = []
# for k in range(len(image_bbox)):
#     img_bbox = []
#     label = []
#     prop = []
#     prediction_list = image_bbox[k].split()
#     i = 0
#     while i < len(prediction_list):
#         if i % 6 == 0:
#             bbox = []
#             label.append(int(prediction_list[i]))
#         elif i % 6 == 1:
#             prop.append(float(prediction_list[i]))
#         else:
#             bbox.append(float(prediction_list[i]))
#         i += 1
#         if i % 6 == 0:
#             img_bbox.append(bbox)
#     whole_bbox.append(list(img_bbox))
#     whole_label.append(list(label))
#     whole_prop.append(list(prop))